In [8]:
import pandas as pd
import re

In [11]:
''' Dados de entrada '''

data_path = '../data/'
projects_file = 'projetos_assembleia.csv'
dialogs_file = 'dialog_lines_with_discussion_topics_EXAMPLE.csv'
#corruption_words = ['corrupto', 'corrupta', 'corrupção']
corruption_words = ['corrup']


In [13]:
projects = pd.read_csv(data_path+projects_file)
dialogs = pd.read_csv(data_path+dialogs_file)

print(projects['title'].unique())
print(dialogs['Subject'].unique())
projects.head()

['Projeto de Resolução 123/VII' 'Projeto de Lei 637/VII'
 'Projeto de Lei 624/VII' 'Projeto de Lei 625/VII'
 'Apreciação Parlamentar 82/VII' 'Projeto de Resolução 122/VII'
 'Projeto de Lei 622/VII' 'Projeto de Lei 621/VII'
 'Projeto de Lei 617/VII' 'Projeto de Lei 620/VII'
 'Projeto de Lei 616/VII' 'Apreciação Parlamentar 96/VII'
 'Projeto de Lei 618/VII' 'Projeto de Lei 629/VII'
 'Projeto de Lei 619/VII' 'Apreciação Parlamentar 92/VII'
 'Projeto de Lei 627/VII' 'Projeto de Lei 664/VII'
 'Projeto de Lei 665/VII' 'Projeto de Lei 631/VII'
 'Projeto de Resolução 121/VII' 'Projeto de Lei 626/VII'
 'Projeto de Lei 652/VII' 'Projeto de Lei 615/VII'
 'Projeto de Lei 632/VII' 'Projeto de Resolução 126/VII'
 'Projeto de Lei 635/VII' 'Projeto de Lei 654/VII'
 'Apreciação Parlamentar 89/VII' 'Apreciação Parlamentar 87/VII'
 'Projeto de Lei 614/VII' 'Projeto de Lei 658/VII'
 'Projeto de Resolução 471/XIV' 'Projeto de Resolução 550/XIV']
['Abertura' 'Declarações políticas'
 'Petição 644/XIII, Proje

,author,description,title,url
0,Luís Marques Mendes (PSD),Sobre a Agenda 2000,Projeto de Resolução 123/VII,https://www.parlamento.pt/ActividadeParlamenta...
1,"Luís Queiró (CDS-PP),Gonçalo Ribeiro da Costa ...",Conselhos municipais de acção social,Projeto de Lei 637/VII,https://www.parlamento.pt/ActividadeParlamenta...
2,"Pimenta Dias (PCP),Joaquim Matias (PCP),João A...",Isenta de emolumentos a apreciação das contas ...,Projeto de Lei 624/VII,https://www.parlamento.pt/ActividadeParlamenta...
3,"Luís Queiró (CDS-PP),Sílvio Rui Cervan (CDS-PP...","Pronuncia, acompanhamento e apreciação pela As...",Projeto de Lei 625/VII,https://www.parlamento.pt/ActividadeParlamenta...
4,Luís Queiró (CDS-PP),"Decreto-lei n° 15/99,de 15 de Janeiro, que""reg...",Apreciação Parlamentar 82/VII,https://www.parlamento.pt/ActividadeParlamenta...


In [14]:
dialogs.head()

,Transcript,Date,Position,Person,Party,Text,Subject
0,DAR-001,17 DE SETEMBRO DE 2020,0,Presidente,NaN,Dirijo um abraço a todos neste regresso dos Pl...,Abertura
1,DAR-001,17 DE SETEMBRO DE 2020,1,Secretária,Maria da Luz Rosinha,"Sr. Presidente, muito boa tarde a todas e a to...",Abertura
2,DAR-001,17 DE SETEMBRO DE 2020,2,Presidente,NaN,"Como os Srs. Deputados estão a verificar, o an...",Abertura
3,DAR-001,17 DE SETEMBRO DE 2020,3,Secretária,Maria da Luz Rosinha,"Sr. Presidente, deram também entrada os Projet...",Abertura
4,DAR-001,17 DE SETEMBRO DE 2020,4,Presidente,NaN,"Muito obrigado, Sr.ª Secretária Maria da Luz R...",Abertura


In [21]:
def find_words(data_frame, fields, words):
    series = [data_frame[field] for field in fields]
    positions = pd.DataFrame()
    for serie in series:
        for word in words:
            positions[serie.name+'_'+word+"_pos"] = serie.str.lower().str.find(word)
    return positions

def get_citations(data_frame, fields, words):
    positions = find_words(data_frame, fields, words)
    out_list = [] # pd.DataFrame(columns = data_frame.columns)
    i = 0
    for ind in positions.index:
        has_found = False
        for field_word in positions:
            if (positions[field_word][ind] != -1):
                has_found = True
                break
        if(has_found):
            #print(data_frame.iloc[ind].values)
            out_list.append(data_frame.iloc[ind].values)
            i = i + 1
    return pd.DataFrame(out_list, columns = data_frame.columns)

dialogs_with_corruption_citations = get_citations(dialogs, ['Text'], corruption_words)
projects_with_corruption_citations = get_citations(projects, ['description'], corruption_words)

print('Diálogos que citam corrupção: ', len(dialogs_with_corruption_citations))
print('Projetos que citam corrupção: ', len(projects_with_corruption_citations))

print(dialogs_with_corruption_citations['Text'][2])
dialogs_with_corruption_citations.head()

#for line in lines:
#    print(line)
#    print('------------')



Diálogos que citam corrupção:  19
Projetos que citam corrupção:  1
Esperei tantos meses, tantos meses pelo tal plano contra a corrupção que a Ministra da Justiça ia apresentar! E lá apresentou. Vejam que, ontem, conhecemos finalmente a realidade deste plano. Cito a Ministra da Justiça: «O grande ponto é a falta de dados e a falta de um sistema harmónico de dimensão preventiva e repressiva». Era isto que faltava, de facto. Olhando para o que precisamos, era aqui que estava a solução. Andámos enganados tanto tempo e estava nestas simples palavras a solução para a corrupção! Mas o que se podia esperar do partido que, um dia, foi liderado pelo Eng.º Guterres, ou que teve grandes nomes, como o de Armando Vara?! Não podíamos esperar muito mais. No meio disto e da inexistência de um plano contra a corrupção, temos um Presidente que diz que temos de apurar: temos de apurar Pedrógão, temos de apurar Tancos, temos de apurar o Novo Banco, temos de apurar os negócios familiares de membros do Gover

,Transcript,Date,Position,Person,Party,Text,Subject,key
0,DAR-001,17 DE SETEMBRO DE 2020,231,Filipa Roseta,PSD,O Prof. António Costa Silva disse hoje de manh...,Declarações políticas,17 DE SETEMBRO DE 2020Declarações políticas
1,DAR-001,17 DE SETEMBRO DE 2020,233,Filipa Roseta,PSD,… e que de cada vez que há um problema e há um...,Declarações políticas,17 DE SETEMBRO DE 2020Declarações políticas
2,DAR-001,17 DE SETEMBRO DE 2020,277,André Ventura,CH,"Esperei tantos meses, tantos meses pelo tal pl...",Declarações políticas,17 DE SETEMBRO DE 2020Declarações políticas
3,DAR-001,17 DE SETEMBRO DE 2020,309,Presidente,Fernando Negrão,"Damos, assim, por terminada a discussão do seg...","Petição 13/XIV, Projeto de Resolução 550/XIV, ...","17 DE SETEMBRO DE 2020Petição 13/XIV, Projeto ..."
4,DAR-001,17 DE SETEMBRO DE 2020,310,André Silva,PAN,"Sr. Presidente, Sr.as e Srs. Deputados: Começo...","Petição 13/XIV, Projeto de Resolução 550/XIV, ...","17 DE SETEMBRO DE 2020Petição 13/XIV, Projeto ..."


In [22]:
def get_related(full_dataframe, citation_dataframe, key_fields):
    citation_dataframe['key'] = citation_dataframe[key_fields[0]]
    full_dataframe['key'] = full_dataframe[key_fields[0]]
    for i in range(1, len(key_fields)):
        citation_dataframe['key'] = citation_dataframe['key'] + citation_dataframe[key_fields[i]]
        full_dataframe['key'] = full_dataframe['key'] + full_dataframe[key_fields[i]]
    topics = citation_dataframe['key'].unique()
    #print(topics)
    to_out = pd.DataFrame(columns = full_dataframe.columns)
    for topic in topics:
        to_out = to_out.append(full_dataframe.loc[full_dataframe['key'] == topic])
    return to_out
    
dialogs_related_to_corruption = get_related(dialogs, dialogs_with_corruption_citations, ['Date', 'Subject'])
print(len(dialogs_related_to_corruption))
print(dialogs_related_to_corruption['key'].unique())
dialogs_related_to_corruption.head()
#print(related)

403
['17 DE SETEMBRO DE 2020Declarações políticas'
 '17 DE SETEMBRO DE 2020Petição 13/XIV, Projeto de Resolução 550/XIV, Projeto de Resolução 546/XIV'
 '18 DE SETEMBRO DE 2020Projeto de Resolução 471/XIV, Projeto de Lei 499/XIV']


,Transcript,Date,Position,Person,Party,Text,Subject,key
5,DAR-001,17 DE SETEMBRO DE 2020,5,Luís Moreira Testa,PS,"Sr. Presidente, Srs. Deputados: Portugal, País...",Declarações políticas,17 DE SETEMBRO DE 2020Declarações políticas
6,DAR-001,17 DE SETEMBRO DE 2020,6,Ana Paula Vitorino,PS,Muito bem!,Declarações políticas,17 DE SETEMBRO DE 2020Declarações políticas
7,DAR-001,17 DE SETEMBRO DE 2020,7,Luís Moreira Testa,PS,Mas também não é de somenos importância que se...,Declarações políticas,17 DE SETEMBRO DE 2020Declarações políticas
8,DAR-001,17 DE SETEMBRO DE 2020,8,Santinho Pacheco,PS,Muito bem!,Declarações políticas,17 DE SETEMBRO DE 2020Declarações políticas
9,DAR-001,17 DE SETEMBRO DE 2020,9,Luís Moreira Testa,PS,"A meio do mês de outubro, o Governo apresentar...",Declarações políticas,17 DE SETEMBRO DE 2020Declarações políticas


In [23]:
def get_related_projects(citations_df, projects_df):
    #print(citations_df['Subject'].unique())
    to_out = pd.DataFrame(columns = projects_df.columns)
    for subject in citations_df['Subject'].unique():
        #print('subject', subject)
        titles = subject.split(', ')
        for title in titles:
            to_out = to_out.append(projects_df.loc[projects_df['title'] == title])
    return to_out

projects_related_to_corruption = get_related_projects(dialogs_with_corruption_citations, projects)
print(len(projects_related_to_corruption))
projects_related_to_corruption.head()
        

2


,author,description,title,url
33,"André Silva (PAN) , Bebiana Cunha (PAN) , Inês...",Recomenda ao Governo que apresente e submeta à...,Projeto de Resolução 550/XIV,https://www.parlamento.pt/ActividadeParlamenta...
32,"Duarte Alves (PCP) , Bruno Dias (PCP) , Antóni...",Recomenda ao Governo a reversão da alienação d...,Projeto de Resolução 471/XIV,https://www.parlamento.pt/ActividadeParlamenta...


In [43]:
def generate_statistics(all_dialogs, direct_citations_dialogs, related_dialogs, projects, direct_citations_projects, related_projects):
    people_statistics = {'name' : [], 
               'amount_of_corruption_citations' : [], 
               'amount_of_discussion_participations' : [],
                'percentage_of_discussion_participations' : []}
    party_statistics = {'name' : [], 
               'amount_of_corruption_citations' : [], 
               'amount_of_discussion_participations' : []}
    projects_statistics = {'total_amount_of_projects' : 0,
                'amount_of_projects_with_direct_citations' : 0,
                'amount_of_related_projects' : 0,
                'percentage_of_corruption_related_projects' : 0.,
                'people_involved' : []}
    # PEOPLE
    people_statistics['name'] = dialogs['Person'].unique()
    total_participations = len(dialogs_related_to_corruption)
    for name in people_statistics['name']:
        people_statistics['amount_of_corruption_citations'].append(len(direct_citations_dialogs.loc[direct_citations_dialogs['Person'] == name]))
        people_statistics['amount_of_discussion_participations'].append(len(dialogs_related_to_corruption.loc[dialogs_related_to_corruption['Person'] == name]))
        people_statistics['percentage_of_discussion_participations'].append(float(people_statistics['amount_of_discussion_participations'][-1])/total_participations)
    # PARTIES
    party_statistics['name'] = dialogs['Party'].unique()
    for name in party_statistics['name']:
        party_statistics['amount_of_corruption_citations'].append(len(direct_citations_dialogs.loc[direct_citations_dialogs['Party'] == name]))
        party_statistics['amount_of_discussion_participations'].append(len(dialogs_related_to_corruption.loc[dialogs_related_to_corruption['Party'] == name]))
    # PROJECTS
    projects_statistics['amount_of_related_projects'] = len(projects_related_to_corruption)
    projects_statistics['amount_of_projects_with_direct_citations'] = len(projects_with_corruption_citations)
    projects_statistics['total_amount_of_projects'] = len(projects)
    projects_statistics['percentage_of_corruption_related_projects'] = float(projects_statistics['amount_of_related_projects'])/projects_statistics['total_amount_of_projects']
    for authors_list in projects_related_to_corruption['author'].unique():
        for author in authors_list.split(','):
            if(author not in projects_statistics['people_involved']):
                projects_statistics['people_involved'].append(author)
    return pd.DataFrame.from_dict(people_statistics), pd.DataFrame.from_dict(party_statistics), projects_statistics
 
people_statistics, party_statistics, projects_statistics = generate_statistics(dialogs, dialogs_with_corruption_citations, dialogs_related_to_corruption, projects, projects_with_corruption_citations, projects_related_to_corruption)

print('PROJETOS')
print(projects_statistics)
print('PESSOAS')
people_statistics.head()

PROJETOS
{'total_amount_of_projects': 34, 'amount_of_projects_with_direct_citations': 1, 'amount_of_related_projects': 2, 'percentage_of_corruption_related_projects': 0.058823529411764705, 'people_involved': ['André Silva (PAN) ', ' Bebiana Cunha (PAN) ', ' Inês de Sousa Real (PAN)', 'Duarte Alves (PCP) ', ' Bruno Dias (PCP) ', ' António Filipe (PCP) ', ' Paula Santos (PCP) ', ' Diana Ferreira (PCP) ', ' João Dias (PCP) ', ' Alma Rivera (PCP) ', ' Jerónimo de Sousa (PCP) ', ' Vera Prata (PCP) ', ' Ana Mesquita (PCP)']}
PESSOAS


,name,amount_of_corruption_citations,amount_of_discussion_participations,percentage_of_discussion_participations
0,Presidente,1,111,0.275434
1,Secretária,0,0,0.000000
2,Luís Moreira Testa,0,15,0.037221
3,Ana Paula Vitorino,0,1,0.002481
4,Santinho Pacheco,0,1,0.002481


In [42]:
print('PARTIDOS')
party_statistics.head()


PARTIDOS


,name,amount_of_corruption_citations,amount_of_discussion_participations
0,NaN,0,0
1,Maria da Luz Rosinha,0,0
2,PS,2,48
3,PEV,2,14
4,CH,4,17
